# TF CNN Classifier

To run this notebook on an another benchmark, use

```
papermill utils/tf_cnn_classifier.ipynb tf_cnn_experiments/[DATASET NAME].ipynb -p DATASET [DATASET NAME]
```

In [1]:
DATASET = 'demo_coding_vs_intergenomic_seqs'
VERSION = 0
BATCH_SIZE = 64
EPOCHS = 10

In [2]:
# Parameters
DATASET = "demo_human_ocr_ensembl_regulatory"


In [3]:
print(DATASET, VERSION, BATCH_SIZE, EPOCHS)

demo_human_ocr_ensembl_regulatory 0 64 10


# Data download

In [4]:
from pathlib import Path
import tensorflow as tf

import numpy as np
from genomic_benchmarks.loc2seq import download_dataset
from genomic_benchmarks.data_check import is_downloaded, info
from genomic_benchmarks.models.tf import vectorize_layer, binary_f1_score
from genomic_benchmarks.models.tf import basic_cnn_model_v0 as model

if not is_downloaded(DATASET):
    download_dataset(DATASET)

2022-02-14 11:45:36.960963: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-02-14 11:45:36.961011: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-02-14 11:45:38.567021: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-02-14 11:45:38.567058: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-02-14 11:45:38.567079: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (katarina-lenovo): /proc/driver/nvidia/version does not exist
2022-02-14 11:45:38.567337: I tensorflow/core/platform/cpu_fe

In [5]:
info(DATASET)

## TF Dataset object

In [6]:
SEQ_PATH = Path.home() / '.genomic_benchmarks' / DATASET
CLASSES = [x.stem for x in (SEQ_PATH/'train').iterdir() if x.is_dir()]

train_dset = tf.keras.preprocessing.text_dataset_from_directory(
    SEQ_PATH / 'train',
    batch_size=BATCH_SIZE,
    class_names=CLASSES)

Found 139804 files belonging to 2 classes.


## Text vectorization

In [7]:
vectorize_layer.adapt(train_dset.map(lambda x, y: x))
# vectorize_layer.set_vocabulary(vocabulary=np.asarray(['a', 'c', 't', 'g', 'n']))
vectorize_layer.get_vocabulary()

['', '[UNK]', 't', 'a', 'g', 'c', 'n']

In [8]:
def vectorize_text(text, label):
  text = tf.expand_dims(text, -1)
  return vectorize_layer(text)-2, label

train_ds = train_dset.map(vectorize_text)

## Model training

In [9]:
model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              optimizer='adam',
              metrics=[tf.metrics.BinaryAccuracy(threshold=0.0), binary_f1_score])

In [10]:
history = model.fit(
    train_ds,
    epochs=EPOCHS)

Epoch 1/10
2185/2185 [==============================] - 77s 34ms/step - loss: 0.6270 - binary_accuracy: 0.6497 - f1_score: 0.4504
Epoch 2/10
2185/2185 [==============================] - 75s 34ms/step - loss: 0.5871 - binary_accuracy: 0.6901 - f1_score: 0.5716
Epoch 3/10
2185/2185 [==============================] - 75s 34ms/step - loss: 0.5669 - binary_accuracy: 0.7065 - f1_score: 0.6122
Epoch 4/10
2185/2185 [==============================] - 75s 34ms/step - loss: 0.5567 - binary_accuracy: 0.7174 - f1_score: 0.6305
Epoch 5/10
2185/2185 [==============================] - 74s 34ms/step - loss: 0.5490 - binary_accuracy: 0.7241 - f1_score: 0.6418
Epoch 6/10
2185/2185 [==============================] - 75s 34ms/step - loss: 0.5435 - binary_accuracy: 0.7281 - f1_score: 0.6498
Epoch 7/10
2185/2185 [==============================] - 75s 34ms/step - loss: 0.5385 - binary_accuracy: 0.7314 - f1_score: 0.6554
Epoch 8/10
2185/2185 [==============================] - 75s 35ms/step - loss: 0.5348 - bin

## Evaluation on the test set

In [11]:
test_dset = tf.keras.preprocessing.text_dataset_from_directory(
    SEQ_PATH / 'test',
    batch_size=BATCH_SIZE,
    class_names=CLASSES)

test_ds =  test_dset.map(vectorize_text)

Found 34952 files belonging to 2 classes.


In [12]:
model.evaluate(test_ds)

547/547 [==============================] - 7s 13ms/step - loss: 0.5403 - binary_accuracy: 0.7309 - f1_score: 0.6825


[0.5403114557266235, 0.7309166789054871, 0.6824724078178406]